In [ ]:
import great_expectations as ge

context = ge.get_context()

In [ ]:
context.datasources

In [ ]:
datasource_name = "sf_datasource"
sf_datasource = context.get_datasource(datasource_name)

In [ ]:
customer_asset_name = "customer_asset"
customer_query = "select * from customer"

customer_asset = sf_datasource.add_query_asset(name=customer_asset_name, query=customer_query, )

In [ ]:
orders_asset_name = "orders_asset"
orders_table = "orders"

orders_asset = sf_datasource.add_table_asset(name=orders_asset_name, table_name=orders_table)

In [ ]:
region_asset_name = "region_asset"
region_table = "region"

region_asset = sf_datasource.add_table_asset(name=region_asset_name, table_name=region_table)

In [ ]:
orders_asset

In [ ]:
customer_asset

In [ ]:
customer_asset.batch_request_options

In [ ]:
orders_asset.batch_request_options

In [ ]:
customer_batch_request = customer_asset.build_batch_request()
customer_batch_request

In [ ]:
orders_batch_request = orders_asset.build_batch_request()
orders_batch_request

In [ ]:
customer_batches = customer_asset.get_batch_list_from_batch_request(customer_batch_request)

for batch in customer_batches:
    print(batch.batch_spec)

In [ ]:
orders_batches = orders_asset.get_batch_list_from_batch_request(orders_batch_request)

for batch in orders_batches:
    print(batch.batch_spec)

In [ ]:
%%markdown

## Adding splitter

In [ ]:
orders_asset

In [ ]:
orders_asset.add_splitter_year_and_month("o_orderdate")

In [ ]:
orders_asset.batch_request_options

In [ ]:
orders_batch_request = orders_asset.build_batch_request()
orders_batch_request

In [ ]:
orders_batches = orders_asset.get_batch_list_from_batch_request(orders_batch_request)

ls = ""
for batch in orders_batches:
    ls = batch
    break

In [84]:
region_asset.add_splitter_column_value

Help on method add_splitter_column_value in module great_expectations.datasource.fluent.sql_datasource:

add_splitter_column_value(column_name: 'str') -> 'Self' method of great_expectations.datasource.fluent.sql_datasource.TableAsset instance
    --Public API--Associates a column value splitter with this sql asset.
    Args:
        column_name: A column name of the column to split on.
    Returns:
        This sql asset so we can use this method fluently.



In [79]:
region_asset.batch_request_options

('r_regionkey',)

In [81]:
region_batch_request = region_asset.build_batch_request({"r_regionkey":1})

In [83]:
region_batches = region_asset.get_batch_list_from_batch_request(region_batch_request)

for batch in orders_batches:
    if batch.batch_spec["data_asset_name"] == region_asset_name:
        print(batch.batch_spec)

In [85]:
%%markdown

## Create Expectation Suite and Expectations


## Create Expectation Suite and Expectations


In [87]:
tpch_suite_name = "tpch_expectation_suite"
tpch_suite = context.add_or_update_expectation_suite(expectation_suite_name= tpch_suite_name)

In [88]:
from great_expectations.expectations.expectation import (
    ExpectationConfiguration,
)

In [91]:
order_custkey_is_not_null = ExpectationConfiguration(
        expectation_type="expect_column_values_to_not_be_null",
    kwargs={
        "column": "o_custkey",
        "mostly": 0.95,
    },
)   

In [93]:
tpch_suite.add_expectation(
    expectation_configuration=order_custkey_is_not_null
)

{"expectation_type": "expect_column_values_to_not_be_null", "kwargs": {"column": "o_custkey", "mostly": 0.95}, "meta": {}}

In [95]:
orders_checkpoint = context.add_or_update_checkpoint(
    name="order_checkpoint",
    validations=[
        {
            "batch_request": orders_batch_request,
            "expectation_suite_name": tpch_suite_name,
        },
    ],
)
    

In [96]:
orders_checkpoint.run()

Calculating Metrics: 0it [00:00, ?it/s]

{
  "run_id": {
    "run_name": null,
    "run_time": "2024-02-15T09:51:37.187400+00:00"
  },
  "run_results": {
    "ValidationResultIdentifier::tpch_expectation_suite/__none__/20240215T095137.187400Z/sf_datasource-orders_asset-year_1995-month_12": {
      "validation_result": {
        "success": true,
        "results": [],
        "evaluation_parameters": {},
        "statistics": {
          "evaluated_expectations": 0,
          "successful_expectations": 0,
          "unsuccessful_expectations": 0,
          "success_percent": null
        },
        "meta": {
          "great_expectations_version": "0.18.8",
          "expectation_suite_name": "tpch_expectation_suite",
          "run_id": {
            "run_name": null,
            "run_time": "2024-02-15T09:51:37.187400+00:00"
          },
          "batch_spec": {
            "type": "table",
            "data_asset_name": "orders_asset",
            "table_name": "orders",
            "schema_name": null,
            "batch_